# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Лабораторная работа 4.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [39]:
import numpy as np
import pandas as pd

reviews=pd.read_csv('reviews_sample.csv', delimiter=',', index_col=0)
recipes=pd.read_csv('recipes_sample.csv', delimiter=',', usecols=['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'])

print(reviews)
print(recipes)

            user_id  recipe_id        date  rating  \
370476        21752      57993  2003-05-01       5   
624300       431813     142201  2007-09-16       5   
187037       400708     252013  2008-01-10       4   
706134   2001852463     404716  2017-12-11       5   
312179        95810     129396  2008-03-14       5   
...             ...        ...         ...     ...   
1013457     1270706     335534  2009-05-17       4   
158736      2282344       8701  2012-06-03       0   
1059834      689540     222001  2008-04-08       5   
453285   2000242659     354979  2015-06-02       5   
691207       463435     415599  2010-09-30       5   

                                                    review  
370476   Last week whole sides of frozen salmon fillet ...  
624300   So simple and so tasty!  I used a yellow capsi...  
187037   Very nice breakfast HH, easy to make and yummy...  
706134   These are a favorite for the holidays and so e...  
312179   Excellent soup!  The tomato flavor is

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [40]:
reviews_sample = reviews.sample(frac=0.05)
recipes_sample = recipes.sample(frac=0.05)

with pd.ExcelWriter('recipes.xlsx') as writer:
    recipes_sample.to_excel(writer, sheet_name='Рецепты', index=False)
    reviews_sample.to_excel(writer, sheet_name='Отзывы', index=False)


3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [41]:

wb = xw.Book('recipes.xlsx')
seconds_assign_column = recipes_sample[['minutes']] * 60
seconds_assign_column.columns = ['seconds_assign']
sh_arr = wb.sheets['Рецепты']
sh_arr.range('H1').options(index=False).value = seconds_assign_column



4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [42]:
fml = xw.Range('I2').formula = f'=D2*60'
xw. Range("I2:I1501").formula = fml
sh_arr.range('I1').value = 'seconds_formula'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [43]:

sh_arr.range('H1:I1').api.Font.Bold = True
sh_arr.range('H:I').api.HorizontalAlignment = -4108


6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [49]:
for cell in sh_arr.range('C2:C1501'):
    if cell.value < 5:
        cell.color = (0, 255, 0)
    elif cell.value > 10:
        cell.color = (255, 0, 0)
    else:
        cell.color = (255, 255, 0)


7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [52]:

vlookup_formula = '=VLOOKUP(B2, Отзывы!B2:B1501, 2, FALSE)'
sh_arr.range('J2').formula = vlookup_formula

countif_formula = '=COUNTIF(Отзывы!B2:B1501, B2)'
sh_arr.range('K2').formula = countif_formula

sh_arr.range('J1').value = 'RecipeId'
sh_arr.range('K1').value = 'n_reviews'
sh_arr.range('J1:K1501').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter


## Лабораторная работа 4.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 